In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [ ]:
# Configurar WebDriver
driver = webdriver.Chrome()

In [ ]:
# Carregar planilha
df = pd.read_excel('jogadores.xlsx')

In [ ]:
# Adicionar colunas de resultado
df['Nacionalidade'] = ''
df['Instagram'] = ''

In [ ]:
for idx, row in df.iterrows():
    nome = row['Nome do Jogador']
    search_url = f"https://www.transfermarkt.com.br/schnellsuche/ergebnis/schnellsuche?query={nome.replace(' ', '+')}"
    
    driver.get(search_url)
    time.sleep(2)  # tempo para carregar a página

    try:
        # Pega o primeiro resultado da busca
        jogador_link = driver.find_element(By.CSS_SELECTOR, 'a.spielprofil_tooltip').get_attribute('href')
        driver.get(jogador_link)
        time.sleep(2)

        # Extrai nacionalidade
        nacionalidade = driver.find_element(By.CSS_SELECTOR, '.dataZusatzDaten img').get_attribute('title')

        # Extrai perfil Instagram, se tiver
        redes_sociais = driver.find_elements(By.CSS_SELECTOR, 'a.socialmedia-icon')
        instagram = ''
        for link in redes_sociais:
            href = link.get_attribute('href')
            if 'instagram.com' in href:
                instagram = href
                break

        # Preenche no DataFrame
        df.at[idx, 'Nacionalidade'] = nacionalidade
        df.at[idx, 'Instagram'] = instagram

        print(f"Coletado: {nome} - {nacionalidade} - {instagram}")

    except Exception as e:
        print(f"Erro ao coletar dados de {nome}: {e}")
        continue

In [ ]:
# Salva os dados na planilha
df.to_excel('jogadores_atualizado.xlsx', index=False)

In [ ]:
driver.quit()
